In [ ]:
# !pip install pandas
# !pip install torch
# !pip install tqdm  # pip install ipywidgets or something
!pip install wandb
!pip install gensim
# !pip install torcheval
!pip install torchmetrics

In [2]:
USING_WANDB = True  # Set to false if not Peter

if USING_WANDB:
    import wandb
    wandb.login()
    # !wandb login

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_folder = "/content/drive/MyDrive/dna_ml_model_data"
accessible_file = f"{data_folder}/accessible.fasta"
reduced_nonaccessible_file = f"{data_folder}/reduced_nonaccessible.fasta"
test_file = f"{data_folder}/test.fasta"
rest_nonaccessible_file = f"{data_folder}/reduced_nonaccessible.fasta"

DATA_ZIP_FILE = "Files.zip"
LINES_PER_SEQUENCE = 4
ACCESSIBLE_LABEL = 1
NOT_ACCESSIBLE_LABEL = 0
TEMP_SPLIT = 0.30  # validation + test
TEST_SPLIT = 0.15
PRETRAINED_DIR = "pretrained"
SOLUTION_FILE = "predictions.csv"

In [55]:
import dna_dataset, utils, CNNModel # , LSTMCNNModel, constants
import torch.nn as nn, torch.optim as optim, torch
from tqdm.notebook import tqdm
import importlib, os, numpy as np, pandas
from datetime import datetime
from sklearn.model_selection import train_test_split
# from torcheval.metrics.functional import multiclass_f1_score
from torchmetrics.functional import f1_score

importlib.reload(dna_dataset)
importlib.reload(utils)
importlib.reload(CNNModel)
# importlib.reload(LSTMCNNModel)
# importlib.reload(constants)

<module 'CNNModel' from '/content/CNNModel.py'>

In [ ]:
# !mkdir Files
# !mv reduced_nonaccessible.fasta Files
# !mv accessible.fasta Files
# !mv test.fasta Files

In [ ]:
# Unzip the zip file
!unzip $DATA_ZIP_FILE

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Read the data from data files

In [8]:
acc_sequences, acc_labels, _ = dna_dataset.read_data_file(accessible_file, accessible=True, shuffle=True)
not_sequences, not_labels, _ = dna_dataset.read_data_file(reduced_nonaccessible_file, accessible=False, shuffle=True)

In [9]:
full_size = len(acc_sequences) + len(not_sequences)

# Split accessible data into train val test split
acc_seq_train, acc_seq_temp, acc_lab_train, acc_lab_temp = train_test_split(acc_sequences, acc_labels, test_size=TEMP_SPLIT, random_state=1)
acc_seq_val, acc_seq_test, acc_lab_val, acc_lab_test = train_test_split(acc_seq_temp, acc_lab_temp, test_size=TEST_SPLIT/TEMP_SPLIT, random_state=1)

# Split not accessible data into train val test split
not_seq_train, not_seq_temp, not_lab_train, not_lab_temp = train_test_split(not_sequences, not_labels, test_size=TEMP_SPLIT, random_state=1)
not_seq_val, not_seq_test, not_lab_val, not_lab_test = train_test_split(not_seq_temp, not_lab_temp, test_size=TEST_SPLIT/TEMP_SPLIT, random_state=1)

# assert(self.accessible_count + self.not_accessible_count == len(self.sequences))
assert(len(acc_sequences) == len(acc_labels) == len(acc_seq_train) + len(acc_seq_val) + len(acc_seq_test))
assert(len(not_sequences) == len(not_labels) == len(not_lab_train) + len(not_lab_val) + len(not_lab_test))

In [43]:
# Create the (shuffled) datasets, after appending the lists
sequences_train = acc_seq_train + not_seq_train
label_train = acc_lab_train + not_lab_train
train_dataset = dna_dataset.DNADataset(sequences_train, label_train, ids=None)

sequences_val = acc_seq_val + not_seq_val
label_val = acc_lab_val + not_lab_val
val_dataset = dna_dataset.DNADataset(sequences_val, label_val, ids=None)

sequences_test = acc_seq_test + not_seq_test
label_test = acc_lab_test + not_lab_test
test_dataset = dna_dataset.DNADataset(sequences_test, label_test, ids=None)

assert(len(sequences_train) + len(sequences_val) + len(sequences_test) == full_size)
# full_dataset = dna_dataset.DNADataset(accessible_file, reduced_nonaccessible_file)

In [46]:
# ensure the DNADataset is loaded properly
print('total sequences', len(acc_sequences) + len(not_sequences))
print('num accessible', len(acc_sequences))
print('num not accessible', len(not_sequences))
i = 0
print(f"example entry {i}")
item = train_dataset[i]
# print(item['sequences'])  # long answer
print("label", item['labels'])
# ensure dataset was shuffled properly
# check that not all the accessible labels are at the front
for i in range(len(train_dataset)):
    if train_dataset[i]['labels'] != ACCESSIBLE_LABEL:
        print('shuffled')
        break

total sequences 94478
num accessible 47239
num not accessible 47239
example entry 0
label 1
shuffled


In [11]:
# # Split dataset
# full_size = len(full_dataset)
# temp_size = round(VALIDATION_SPLIT * 2 * full_size)  # * 2 for both validation and test split
# train_size = full_size - temp_size

# train_dataset, temp_dataset = torch.utils.data.random_split(full_dataset, [train_size, temp_size])

# val_size = temp_size // 2
# test_size = temp_size - val_size
# assert(val_size + test_size == temp_size)

# val_dataset, test_dataset = torch.utils.data.random_split(temp_dataset, [val_size, test_size])

# print(len(train_dataset), len(val_dataset), len(test_dataset))
# assert(len(train_dataset) + len(val_dataset) + len(test_dataset) == full_size)

66135 14171 14172


Upsample or Downsample the dataset  
(AFTER splitting to train/validation/test to prevent data leakage)

In [78]:
importlib.reload(CNNModel)
torch.manual_seed(0)

kernel_size = 3     # 2  # should use odd size
embed_dim = 4  # 4
num_filters1 = 64  # 128
num_filters2 = 128  # 64
pool_kernel_size = 2  # 2
hidden_dense1 = 64  # 128
hidden_dense2 = 32  # 64
dropout_rate_Dense = 0.5  # .5

# insert torch model here, that takes sequence as input and output a label 0 or 1
model = CNNModel.CNNModel(kernel_size,
                           embed_dim,
                           num_filters1,
                           num_filters2,
                           pool_kernel_size,
                           hidden_dense1,
                           hidden_dense2,
                           dropout_rate_Dense
                           )

# importlib.reload(LSTMCNNModel)
# model = LSTMCNNModel.LSTMCNNModel(
#                             kernel_size=2,
#                             embed_dim=4,
#                             num_filters1=128,
#                             num_filters2=64,
#                             pool_kernel_size=2,
#                             hidden_dense1=128,
#                             hidden_dense2=64,
#                             dropout_rate_Dense=0.5,
#                             lstm_units=1
# )

model.to(device)

CNNModel(
  (Conv1): Conv1d(4, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=6400, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=32, bias=True)
  (linear3): Linear(in_features=32, out_features=1, bias=True)
  (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout_Dense): Dropout(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)

In [79]:
# parameters
epochs = 20
batch_size = 64
learning_rate = .0001
loss_fn = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)  # weight_decay=1
# optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=.01)  # weight_decay=1
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)  # pretty bad

In [80]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, # shuffle=True
)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, # shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, # shuffle=True
)

In [81]:
USING_WANDB = True

In [82]:
# TRAINING LOOP
# USING_WANDB = False
if USING_WANDB:
    note = f"testing dataset rework"
    # note = f"{optimizer.__class__.__name__}-{kernel_size}-kernel-{num_filters1}-{num_filters2}-conv-{hidden_dense1}-{hidden_dense2}-dense" # "lstm_no_dropout" # "0.5_dropout"
    wandb.init(
        project="dna_ml_model",
        # name=f"experiment_{run}"
        name=f"{note}",
        settings=wandb.Settings(start_method="fork"),
        config = {
            "architecture": model.__class__.__name__,
            "epochs": epochs,
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "optimizer": optimizer.__class__.__name__,
            "notes": note
    })


# step = 0
best_acc = 0.0

for epoch in range(epochs):
    print(f"Epoch {epoch + 1} of {epochs}")

    running_loss = 0.0
    running_corrects = 0.0
    total_outputs = torch.empty(0).to(device)
    total_labels = torch.empty(0).to(device)

    model.train()
    for batch in tqdm(train_loader):  # show the times for each batch
        # Forward propagate
        samples, labels = batch["sequences"].to(device), batch['labels'].to(device)

        outputs = model(samples).flatten()

        labels = labels.reshape(-1,1).float().flatten()

        # Backpropagation and gradient descent
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()  # reset gradients before next iteration

        running_loss += loss.item() * samples.size(0)  # loss per sample times batch size

        preds = torch.round(outputs)
        running_corrects += torch.sum(preds == labels.data).item()

        total_outputs = torch.cat((total_outputs, outputs))
        total_labels = torch.cat((total_labels, labels))

    train_f1 = f1_score(total_outputs.flatten(), total_labels.flatten(), task="binary", num_classes=2).item()


    # if step % n_eval == 0:
    # Compute training loss and accuracy.
    with torch.no_grad():
        model.eval()
        # Compute validation loss and accuracy.
        # accuracy = utils.compute_accuracy(outputs, labels)  # only does current batch
        val_loss, val_acc, val_f1 = utils.evaluate(val_loader, model, loss_fn, device)

        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects / len(train_dataset)
        print(f"Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} F1: {train_f1:.4f}")
        print(f"Validation Loss: {val_loss:.4f} Acc: {val_acc:.4f} F1: {val_f1:.4f}")

        # # deep copy the model
        # if val_accuracy > best_acc:
        #     best_acc = epoch_acc
        #     torch.save(model.state_dict(), best_model_params_path)

        if USING_WANDB:
            wandb.log({"Train Loss": epoch_loss,
                        "Train Acc": epoch_acc,
                        "Val Loss": val_loss,
                        "Val Acc": val_acc,
                       "Train F1": train_f1,
                       "Val F1": val_f1,
                      #  "Epoch": epoch
            })
        # step += 1

    print()

Epoch 1 of 20


  0%|          | 0/1034 [00:00<?, ?it/s]

Training Loss: 0.6415 Acc: 0.5942 F1: 0.4634
Validation Loss: 130.3700 Acc: 0.7163 F1: 0.7197

Epoch 2 of 20


  0%|          | 0/1034 [00:00<?, ?it/s]

Training Loss: 0.6227 Acc: 0.6093 F1: 0.4875
Validation Loss: 126.6128 Acc: 0.7154 F1: 0.6995

Epoch 3 of 20


...

Training Loss: 0.5435 Acc: 0.6596 F1: 0.5493
Validation Loss: 107.4620 Acc: 0.7724 F1: 0.7679

Epoch 19 of 20


  0%|          | 0/1034 [00:00<?, ?it/s]

Training Loss: 0.5431 Acc: 0.6631 F1: 0.5554
Validation Loss: 107.6594 Acc: 0.7715 F1: 0.7684

Epoch 20 of 20


  0%|          | 0/1034 [00:00<?, ?it/s]

Training Loss: 0.5432 Acc: 0.6622 F1: 0.5535
Validation Loss: 107.1393 Acc: 0.7659 F1: 0.7545



In [51]:
# wandb.finish()

In [85]:
# EVALUATE on test dataset
total_predictions = 0
total_correct = 0

model.eval()

for batch in test_loader:  # tqdm()
    test_samples, test_labels = batch['sequences'].to(device), batch['labels'].to(device)
    test_outputs = model(test_samples)
    # print(torch.round(val_outputs))
    test_labels = test_labels.reshape(-1, 1).float()

    test_loss = loss_fn(test_outputs, test_labels).item()  # change tensor to single val

    total_correct += (torch.round(test_outputs) == test_labels).sum().item()

    total_predictions += len(test_outputs)

model.train() # here
test_accuracy = total_correct / total_predictions
print(f"Final Test Accuracy: {test_accuracy * 100} %")

wandb.summary['test_accuracy'] = test_accuracy

Final Test Accuracy: 76.2489415749365 %


In [53]:
# test on rest of nonaccessible
rest_non_sequences, rest_non_labels, _ = dna_dataset.read_data_file(rest_nonaccessible_file, accessible=False, shuffle=True)

rest_nonacc_dataset = dna_dataset.DNADataset(rest_non_sequences, rest_non_labels, _)
rest_nonacc_loader = torch.utils.data.DataLoader(
    rest_nonacc_dataset, batch_size=batch_size, # shuffle=True
)

In [90]:
# test on rest of nonaccessible
probs_out = []  # tuples of probability, id

# here

for batch in rest_nonacc_loader:  # tqdm()
    samples, _ = batch["sequences"].to(device), batch['ids'] # not a tensor
    outputs = model(samples)
    out_list = outputs.tolist()

    for i in range(len(out_list)):
        probs_out.append(out_list[i])

probs_out = np.array(probs_out)
falses = len(probs_out[probs_out<0.5])
print("correctly predicted ", falses, " non_accessible values out of ", len(probs_out), " total")
rest_non_correct = falses / len(probs_out)

print(rest_non_correct, " correct (all not accessible)")

wandb.summary['rest_nonacc_accuracy'] = rest_non_correct

wandb.finish()

correctly predicted  40592  non_accessible values out of  47239  total
0.8592899934376257  correct (all not accessible)


Save Model

In [54]:
# Create pretrained directory if not yet created
if not os.path.isdir(PRETRAINED_DIR):
    os.mkdir(PRETRAINED_DIR)

now = datetime.now()
datetime_str = now.strftime("%m-%d-%H-%M-%S")
model_save_path = os.path.join(
    PRETRAINED_DIR,
    f'{note}{datetime_str}-{model.__class__.__name__}-model-{learning_rate}lr-{epochs}epochs.pt'
)
print('model_save_path', model_save_path)
CNNModel.save_CNNModel(model_save_path, model)
print(f"model saved at {datetime_str}")

model_save_path pretrained/AdamW-3-kernel-64-128-conv-64-32-dense01-02-21-24-36-CNNModel-model-0.0001lr-20epochs.pt
model saved at 01-02-21-24-36


Load Model

In [ ]:
model_save_path = "/content/pretrained/12-10-05-30-17-CNNModel-model-10-epochs.pt"
model = CNNModel.load_CNNModel(model_save_path)
model.to(device)

Inference on Competition File

In [56]:
# Load Competitation Data
comp_sequences, _, comp_ids = dna_dataset.read_data_file(test_file, labeled=False, shuffle=True)

competition_dataset = dna_dataset.DNADataset(comp_sequences, labels=None, ids=comp_ids)
competition_loader = torch.utils.data.DataLoader(
    competition_dataset, batch_size=batch_size, # shuffle=True
)
print("Competition dataset loaded.")

Competition dataset loaded.


In [91]:
probs = []  # tuples of probability, id

# here
for batch in competition_loader:  # tqdm()

    samples, ids = batch["sequences"].to(device), batch['ids']  # not a tensor

    outputs = model(samples)

    out_list = outputs.tolist()

    for i in range(len(out_list)):
        probs.append((out_list[i], ids[i]))

print("Finished inference")

  0%|          | 0/4209 [00:00<?, ?it/s]

Finished inference


In [94]:
# View the results
np_probs = np.array(list(zip(*probs))[0])
print("Predicted", len(np_probs[np_probs>0.5]), "true values out of ", len(np_probs), " total")
not_zero = np_probs[np_probs<=0.5]
not_zero.sort()
not_one = np_probs[np_probs>0.5]
not_one[::-1].sort()
print("not accessible probs", not_zero)
print("accessible probs", not_one)

# print(np_probs[np_probs>0.0 and np_probs<1.0])

Predicted 68348 true values out of  269315  total
not accessible probs [1.85848257e-05 2.12803388e-05 3.71750757e-05 ... 4.99978244e-01
 4.99986887e-01 4.99989390e-01]
accessible probs [0.9951238  0.99226063 0.99152654 ... 0.50000709 0.50000608 0.50000435]


In [98]:
probs.sort(reverse=True)
print("first 10\n", list(zip(*probs[:10]))[0])
print("last 10 of top 10000\n", list(zip(*probs[9990:10000]))[0])  # probs only

highest_probs = probs[:10000]  # top 10,000

with open(SOLUTION_FILE, "w") as f:
    for pair in highest_probs:
        f.write(pair[1])
        # f.write("a")
        f.write("\n")

first 10
 ([0.9951238036155701], [0.9922606348991394], [0.9915265440940857], [0.9914847016334534], [0.9913023710250854], [0.9894152879714966], [0.9892381429672241], [0.9892197251319885], [0.9891876578330994], [0.9885799288749695])
last 10 of top 10000
 ([0.8017553091049194], [0.8017517924308777], [0.8017512559890747], [0.8017467260360718], [0.8017371892929077], [0.8017303943634033], [0.8017300963401794], [0.8017240762710571], [0.801720917224884], [0.8017192482948303])


In [99]:
zip_file_name = "predictions.zip"
!zip $zip_file_name $SOLUTION_FILE

  adding: predictions.csv (deflated 65%)


In [ ]:
# ONLY for use on google colab. download files
from google.colab import files
import os
dir = 'pretrained'
model_file = os.path.join(dir, os.listdir(dir)[0])
files.download(model_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>